In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Abrir o arquivo DEM
with rasterio.open('2024-05-06-00:00_2024-05-06-23:59_DEM_MAPZEN_DEM_(Raw).tif') as src:
    dem_data = src.read(1)  # Lê a primeira banda

# Mostrar a imagem DEM
plt.imshow(dem_data, cmap='terrain')
plt.colorbar(label='Elevação (m)')
plt.title('Mapa de Elevação')
plt.show()


In [ ]:
# Definir o limite de elevação para áreas de risco
limite_elevacao = 100  # ajuste este valor conforme necessário

# Criar máscara de áreas de risco (elevação abaixo do limite)
areas_risco = dem_data < limite_elevacao

# Visualizar áreas de risco
plt.imshow(areas_risco, cmap='Reds')
plt.title('Zonas de Risco de Enchentes')
plt.show()


In [ ]:
def process_zone(zone, base_path, output_base_path, water_threshold):
    """
    Processa os dados geográficos para uma zona específica.
    
    Args:
        zone (str): Código da zona (ex: 'ZN', 'SN').
        base_path (str): Caminho base onde os arquivos estão armazenados.
        output_base_path (str): Caminho base para salvar os resultados.
        water_threshold (float): Limiar de altitude para áreas aquáticas.
    """
    # Construir os caminhos de entrada e saída com base na zona
    altitude_path = f"{base_path}/{zone}/{zone}.tif"
    risk_index_path = f"{base_path}/{zone}/risk_index.tif"
    output_path = f"{output_base_path}/{zone}_risk_index_masked.tif"
    output_contours_path = f"{output_base_path}/{zone}_altitude_contours.tif"
    
    try:
        # Carregar dados de altitude
        altitude_ds = load_raster_data(altitude_path)
        altitude_data = altitude_ds.GetRasterBand(1).ReadAsArray()
        
        # Aplicar máscara de terra
        land_mask = apply_land_mask(altitude_data, water_threshold)
        
        # Carregar e aplicar máscara ao índice de risco
        risk_ds = load_raster_data(risk_index_path)
        risk_data = risk_ds.GetRasterBand(1).ReadAsArray()
        risk_data.SetGeoTransform(altitude_ds.GetGeoTransform())
        risk_data.SetProjection(altitude_ds.GetProjection())
        risk_data_masked = np.where(land_mask, risk_data, np.nan)
        
        # Salvar dados de risco processados
        save_raster_data(output_path, risk_ds, risk_data_masked, np.nan)
        
        # Gerar e salvar contornos
        save_contours(altitude_ds, land_mask, output_contours_path)
        
        logging.info(f"Processamento concluído para a zona: {zone}")
    except Exception as e:
        logging.error(f"Erro ao processar a zona {zone}: {e}")

# Configurações e chamada da função
base_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas"
output_base_path = "/home/dv/Labs/notebook/sentinel-cloud/data-out"
water_threshold = 6.5

zones = ['ZN', 'SN', 'ON', 'RS', 'FT', 'DD', 'VN', 'HN']
for zone in zones:
    process_zone(zone, base_path, output_base_path, water_threshold)
